In [2]:
import sys
sys.path.insert(0, '..')
sys.path.insert(0, 'plots')

import os

from IPython.display import display
from IPython.display import clear_output
import ipywidgets as widgets


import json
from SplunkIntelOptimized import SplunkIntelOptimized
from sources.SplunkDatasetNew import SplunkDatasetNew
from sources.SplunkFileSource import SplunkFileSource

import plotly as py
import pandas as pd
import numpy as np
import plotNew as plot

py.offline.init_notebook_mode()
pd.set_option('display.notebook_repr_html', True)

data_sources_names = []

data_source_picker = None
time_range_picker = None
event_picker = None
event_text_area = None 
threshold_picker = None
new_time_range_picker = None

#create widgets
def create_widgets():
    global data_source_picker, time_range_picker, event_picker
    global event_text_area, threshold_picker, new_time_range_picker
    for file in os.listdir("../../data"):
            if file.endswith(".json"):
                data_sources_names.append(file)

    data_source_picker = widgets.Dropdown(
            options=data_sources_names,
            description='Data Source:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )    

    time_range_picker = widgets.IntRangeSlider(
                value=[1, 2],
                min=0,
                max=100,
                step=1,
                description='Control Time:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='i',
                slider_color='white',
                color='black'
        )
    
    new_time_range_picker = widgets.IntRangeSlider(
                value=[1, 2],
                min=0,
                max=100,
                step=1,
                description='Test Time:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='i',
                slider_color='white',
                color='black'
        )
    
    threshold_picker = widgets.FloatSlider(
                value=0.8,
                min=0,
                max=1,
                step=0.1,
                description='Threshold:',
                disabled=False,
                continuous_update=False,
                orientation='horizontal',
                readout=True,
                readout_format='.1f',
                slider_color='white',
                color='black'
        )
    
    event_picker = widgets.Dropdown(
            options=[1,2,3],
            description='Event Number:',
            disabled=False,
            button_style='' # 'success', 'info', 'warning', 'danger' or ''
        )
    
    l = widgets.Layout(height='40px', width='800px')
    event_text_area = widgets.Textarea(value='TA: height=40px', layout=l)

    
def initialize(change):    
    # Just to set the widgets
    all_events = []
    all_events.extend(SplunkFileSource.load_data('../../data/' + data_source_picker.value))    
    minutes = 0
    for dict in all_events:
        if dict.get('cluster_label') == '1':
            minutes = minutes + 1

    time_range_picker.value = [1,1]
    time_range_picker.min = 1
    time_range_picker.max = minutes
    
    #new_time_range_picker.value = [minutes -2, minutes]
    new_time_range_picker.value = [1, 1]
    new_time_range_picker.min = 1
    new_time_range_picker.max = minutes 
    
    change_event_handler(None)
    
    
splunkDataset = SplunkDatasetNew()
splunkIntelargs = []

def event_picker_change_handler(change):
    event_text_area.value = json.dumps(splunkDataset.get_all_events_for_notebook()[event_picker.value])

def split(input, length, size):
    input.replace('\n', ' ')
    input.replace('\tat', ' ')
    return '<br>'.join([input[start:start + size] for start in range(0, length, size)])

def get_tool_tips(all_events):
    tooltips = []
    for idx, event in enumerate(all_events):
        tooltips.append(
            split(event[0], min(100, len(event[0])), 100) + '<br> id = ' + str(idx) + '<br> cluster = ' + str(
            event[3]))
            
    return tooltips
    
# Handle data source change
def change_event_handler(change):
    global splunkDataset
    
    control_start = time_range_picker.value[0]
    test_start = new_time_range_picker.value[0]

    clear_output()

    prev_out_file = None
    while(control_start <= time_range_picker.value[1] or test_start < new_time_range_picker.value[1]):

        del splunkIntelargs[:]
        splunkDataset = SplunkDatasetNew() 

        print(control_start, control_start)
        print(test_start, test_start)
        splunkDataset.load_from_file('../../data/' + data_source_picker.value,
                                     [control_start, control_start],
                                     [test_start, test_start], prev_out_file)
    
        splunkIntelargs.append('--sim_threshold')
        splunkIntelargs.append(str(float(threshold_picker.value)))
    
        print(splunkIntelargs)

        splunkIntel = SplunkIntelOptimized(splunkDataset, SplunkIntelOptimized.parse(splunkIntelargs))
        splunkDataset = splunkIntel.run()
    
        file_object  = open("result.json", "w")
        file_object.write(splunkDataset.get_output_as_json)
        file_object.close()
        
        prev_out_file = './result.json'
        
        control_start = control_start + 1
        test_start = test_start + 1

    xy_matrix, tooltips, labels,sizes = splunkDataset.control_scatter_plot()
    new_sizes = []
    for size in sizes:
        new_sizes.append((2 + len(str(size)))**2)
        
    print(new_sizes)
    plot.scatter_plot_groups(xy_matrix, labels, tooltips, ['control', 'test', 'test-anomaly', 'test-unx-freq'], ['blue','green','red', 'orange'])
    
    #xy_matrix, tooltips, labels, sizes = splunkDataset.count_scatter_plot()
    
    #new_sizes = []
    #for size in sizes:
     #   new_sizes.append((3 + len(str(size)))**2)
        
    #print(new_sizes)    
    
    #plot.scatter_plot_groups(xy_matrix, labels, tooltips, ['test', 'test-anomaly'], ['green','red'], new_sizes)


    #for key,data in splunkDataset.get_anom_clusters().items():
     #   for host,anom in data.items():
     #       print(host, anom.get('text'))
    
    xy_matrix, tooltips, labels, clusters = splunkDataset.control_scatter_plot_4d()
    plot.scatter_plot_groups_4d(xy_matrix, labels, clusters, tooltips, ['blue','orange','red'])
            
    event_picker.options = [ x for x in range(len(splunkDataset.get_all_events_for_notebook()))] 
     

create_widgets()

#intialize
initialize(None)
    
#setup even handlers    
data_source_picker.observe(initialize, names="value")
time_range_picker.observe(change_event_handler, names="value")
new_time_range_picker.observe(change_event_handler, names="value")
event_picker.observe(event_picker_change_handler, names="value")
threshold_picker.observe(change_event_handler, names="value")



# show widgets
display(widgets.HBox([data_source_picker]))
display(widgets.HBox([time_range_picker, threshold_picker]))
display(widgets.HBox([new_time_range_picker]))
display(widgets.HBox([event_picker, event_text_area]))

2017-06-28 13:32:07,215 INFO loading file ../../data/mzs.json


(1, 1)
(1, 1)


2017-06-28 13:32:07,465 INFO Running using file source
2017-06-28 13:32:07,466 INFO setting min_df = 0.05 and max_df = 0.9
2017-06-28 13:32:07,467 INFO Start vectorization....


['--sim_threshold', '0.8']


2017-06-28 13:32:10,060 INFO Running kemans with k = 50
2017-06-28 13:32:10,659 INFO found k = 50
2017-06-28 13:32:10,660 INFO Running kemans with k = 25
2017-06-28 13:32:11,024 INFO Running kemans with k = 37
2017-06-28 13:32:11,480 INFO found k = 37
2017-06-28 13:32:11,481 INFO Running kemans with k = 31
2017-06-28 13:32:11,879 INFO found k = 31
2017-06-28 13:32:11,880 INFO Running kemans with k = 28
2017-06-28 13:32:12,246 INFO found k = 28
2017-06-28 13:32:12,247 INFO Running kemans with k = 26
2017-06-28 13:32:12,620 INFO Running kemans with k = 27
2017-06-28 13:32:12,971 INFO found k = 27
2017-06-28 13:32:21,694 INFO Detect Count Anomalies....
2017-06-28 13:32:21,695 INFO Using ThreeSigmaClassifier for cluster 0
2017-06-28 13:32:21,697 INFO Using ThreeSigmaClassifier for cluster 1
2017-06-28 13:32:21,699 INFO Using ZeroDeviationClassifier for cluster 2
2017-06-28 13:32:21,701 INFO Using ThreeSigmaClassifier for cluster 3
2017-06-28 13:32:21,704 INFO Using ZeroDeviationClassifier 

[u'1', u'5', u'1', u'1', u'2', u'1', u'1', u'1', u'1', u'1', u'1']
[u'5']
[-1]
[u'1', u'2', u'1', u'1', u'1', u'1', u'1', u'1', u'1', u'1', u'1', u'1', u'1']
[u'2']
[-1]
[25, 36, 36, 36, 36, 36, 36, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 16, 16, 16, 16, 16, 9, 16, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 16, 9, 9, 9, 9, 9, 9, 36, 36, 9, 25, 25, 9, 9, 36, 36, 9, 9, 9, 9, 16, 16, 16, 16, 16, 16, 16, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 16, 16, 25, 25, 36, 36, 25, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 16, 16, 16, 9, 9, 16, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 25, 36, 9, 25, 25, 9, 9, 16, 36, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 16, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
